In [1]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 自定义训练和批量预测

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/sdk-custom-image-classification-batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fcustom%2Fsdk-custom-image-classification-batch.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab企业版中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/custom/sdk-custom-image-classification-batch.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/sdk-custom-image-classification-batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

## 概述

本教程演示了如何使用Python的Vertex AI SDK来训练和部署自定义图像分类模型以进行批量预测。

了解更多关于[自定义训练](https://cloud.google.com/vertex-ai/docs/training/custom-training)和[Vertex AI批量预测](https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/get-batch-predictions)的信息。

### 目标

在本教程中，您将学习如何使用Vertex AI Training创建一个自定义训练模型，并使用Vertex AI Batch Prediction对训练模型进行批量预测。

使用Vertex AI SDK for Python在Docker容器中从Python脚本创建一个自定义训练模型，然后通过发送数据对部署的模型进行预测。 或者，您可以使用`gcloud`命令行工具或在Cloud控制台上在线创建自定义训练模型。

本教程使用以下Google Cloud ML服务：

- Vertex AI Training
- Vertex AI Batch Prediction
- Vertex AI Model资源

执行的步骤包括：

- 创建一个用于训练TensorFlow模型的Vertex AI自定义作业。
- 将训练模型工件上传为模型资源。
- 进行批量预测。

### 数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[cifar10数据集](https://www.tensorflow.org/datasets/catalog/cifar10)。您将使用的数据集版本已内置于TensorFlow中。训练模型可以预测图像属于十个类别中的哪一类：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船、卡车。

### 费用

本教程使用谷歌云（GCP）的计费组件：

* Vertex AI
* 云存储

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing 计算器](https://cloud.google.com/products/calculator/)来根据您的预期使用情况生成成本估算。

开始吧

### 为Python安装Vertex AI SDK和其他必需的软件包

In [ ]:
! pip3 install --upgrade google-cloud-aiplatform \
                        google-cloud-storage \
                        pillow  \
                        numpy    

### 重新启动运行时（仅限Colab）

为了使用新安装的包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核将重新启动。请等待直到重新启动完成后再继续下一步。⚠️</b>
</div>

### 验证您的笔记本环境（仅适用于Colab）

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置谷歌云项目信息并初始化用于 Python 的 Vertex AI SDK

要开始使用 Vertex AI，您必须拥有一个现有的谷歌云项目并[启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储存储桶

创建一个存储桶来存储中间产物，例如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶还不存在：运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

#### 初始化顶点 AI SDK for Python

为您的项目初始化顶点 AI SDK for Python。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### 导入库并定义常量

In [ ]:
import os

### 设置预构建容器

Vertex AI提供预构建容器以运行训练和预测。

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)和[用于预测的预构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)。

In [ ]:
TRAIN_VERSION = "tf-cpu.2-9"
DEPLOY_VERSION = "tf2-cpu.2-9"

TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(DEPLOY_VERSION)

# 教程

现在您已经准备好开始使用CIFAR10创建自己的自定义训练模型。

训练模型

您可以使用容器镜像的两种方式来训练一个定制模型：

- **使用Google Cloud的预构建容器**。如果您使用预构建的容器，您还需要指定一个要安装到容器镜像中的Python包。这个Python包包含了您用于训练自定义模型的代码。

- **使用您自己的定制容器镜像**。如果您使用自己的容器，容器需要包含您的用于训练自定义模型的代码。

### 定义训练脚本的命令参数

准备要传递给训练脚本的命令行参数。
- `args`：要传递给相应Python模块的命令行参数。在本例中，它们将是：
  - `"--epochs=" + EPOCHS`：训练的周期数。
  - `"--steps=" + STEPS`：每个周期的步骤（批次）数。
  - `"--distribute=" + TRAIN_STRATEGY"`：用于单个或分布式训练的训练分发策略。
     - `"single"`：单个设备。
     - `"mirror"`：所有GPU设备在单个计算实例上。
     - `"multi"`：所有GPU设备在所有计算实例上。

In [ ]:
JOB_NAME = "custom_job_unique"
MODEL_DIR = "{}/{}".format(BUCKET_URI, JOB_NAME)


TRAIN_STRATEGY = "single"

EPOCHS = 20
STEPS = 100

CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--steps=" + str(STEPS),
    "--distribute=" + TRAIN_STRATEGY,
]

培训脚本

在下一个单元格中，您将编写培训脚本`task.py`的内容。总结如下：

- 从环境变量`AIP_MODEL_DIR`获取保存模型工件的目录。这个变量是由训练服务设置的。
- 从TF数据集（tfds）加载CIFAR10数据集。
- 使用TF.Keras模型API构建模型。
- 编译模型（`compile()`）。
- 根据参数`args.distribute`设置训练分布策略。
- 根据参数`args.epochs`和`args.steps`来训练模型（`fit()`）。
- 将已训练的模型保存（`save(MODEL_DIR)`）到指定的模型目录中。

In [ ]:
%%writefile task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64

def make_datasets_unbatched():
  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label

  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model

# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
MODEL_DIR = os.getenv("AIP_MODEL_DIR")

train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(MODEL_DIR)

### 训练模型

在Vertex AI上定义您的自定义训练作业。

使用`CustomTrainingJob`类来定义作业，它接受以下参数：

- `display_name`：此训练流水线的用户定义名称。
- `script_path`：训练脚本的本地路径。
- `container_uri`：训练容器镜像的URI。
- `requirements`：脚本的Python软件包依赖项列表。
- `model_serving_container_image_uri`：可以为您的模型提供预测的容器的URI - 可以是预构建的容器或自定义容器。

使用`run`函数开始训练，它接受以下参数：

- `args`：要传递给Python脚本的命令行参数。
- `replica_count`：工作器复制品的数量。
- `model_display_name`：如果脚本生成托管的`Model`，则为`Model`的显示名称。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作器复制品的加速器数量。

`run`函数创建一个训练流水线，用于训练并创建`Model`对象。训练流水线完成后，`run`函数将返回`Model`对象。

In [ ]:
job = aiplatform.CustomTrainingJob(
    display_name=JOB_NAME,
    script_path="task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["tensorflow_datasets==1.3.0"],
    model_serving_container_image_uri=DEPLOY_IMAGE,
)

MODEL_DISPLAY_NAME = "model_unique"

# Start the training

model = job.run(
    model_display_name=MODEL_DISPLAY_NAME,
    args=CMDARGS,
    replica_count=1,
)

发出一个批量预测请求给你部署的模型。

获取测试数据

从CIFAR数据集下载图像并预处理。

下载测试图像

从CIFAR数据集下载提供的一组图像。

In [ ]:
# Download the images
! gsutil -m cp -r gs://cloud-samples-data/ai-platform-unified/cifar_test_images .

#### 预处理图像
在您能够将数据传递到端点之前，您需要对其进行预处理，以匹配您在`task.py`中定义的自定义模型所期望的格式。

`x_test`：
通过将每个像素除以255来归一化（重新缩放）像素数据。这将使用 32 位浮点数代替每个单字节整数像素，范围在0到1之间。

`y_test`：
您可以从图像文件名中提取标签。每个图像的文件名格式为"image_{LABEL}_{IMAGE_NUMBER}.jpg"

In [ ]:
import numpy as np
from PIL import Image

# Load image data
IMAGE_DIRECTORY = "cifar_test_images"

image_files = [file for file in os.listdir(IMAGE_DIRECTORY) if file.endswith(".jpg")]

# Decode JPEG images into numpy arrays
image_data = [
    np.asarray(Image.open(os.path.join(IMAGE_DIRECTORY, file))) for file in image_files
]

# Scale and convert to expected format
x_test = [(image / 255.0).astype(np.float32).tolist() for image in image_data]

# Extract labels from image name
y_test = [int(file.split("_")[1]) for file in image_files]

准备批量预测的数据
在您可以运行数据进行批量预测之前，您需要将数据保存成几种可能的格式之一。

对于本教程，请使用JSONL，因为它与每个图像目前所在的三维列表兼容。要做到这一点：

1. 在一个文件中，将每个实例作为单独的JSON写入一行。
2. 将此文件上传到云存储。

有关批量预测输入格式的更多详细信息：https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions#batch_request_input

In [ ]:
import json

BATCH_PREDICTION_INSTANCES_FILE = "batch_prediction_instances.jsonl"

BATCH_PREDICTION_GCS_SOURCE = (
    BUCKET_URI + "/batch_prediction_instances/" + BATCH_PREDICTION_INSTANCES_FILE
)

# Write instances at JSONL
with open(BATCH_PREDICTION_INSTANCES_FILE, "w") as f:
    for x in x_test:
        f.write(json.dumps(x) + "\n")

# Upload to Cloud Storage bucket
! gsutil cp $BATCH_PREDICTION_INSTANCES_FILE $BATCH_PREDICTION_GCS_SOURCE

print("Uploaded instances to: ", BATCH_PREDICTION_GCS_SOURCE)

发送预测请求

要发起批量预测请求，调用模型对象的`batch_predict`方法，使用以下参数：
- `instances_format`：批量预测请求文件的格式："jsonl"、"csv"、"bigquery"、"tf-record"、"tf-record-gzip"或"file-list"
- `prediction_format`：批量预测响应文件的格式："jsonl"、"csv"、"bigquery"、"tf-record"、"tf-record-gzip"或"file-list"
- `job_display_name`：预测作业的可读名称。
- `gcs_source`：您的批量预测请求的一个或多个云存储路径列表。
- `gcs_destination_prefix`：服务将写入预测结果的云存储路径。
- `model_parameters`：用于提供预测结果的其他过滤参数。
- `machine_type`：用于训练的计算机类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作副本的加速器数量。
- `starting_replica_count`：最初提供的计算实例数量。
- `max_replica_count`：要扩展到的最大计算实例数量。在本教程中，只提供一个实例。

### 计算实例扩展

您可以指定单个实例（或节点）来处理批量预测请求。本教程使用单个节点，因此变量`MIN_NODES`和`MAX_NODES`均设置为`1`。

如果要使用多个节点来处理批量预测请求，请将`MAX_NODES`设置为您要使用的最大节点数。Vertex AI会自动调整用于提供预测的节点数量，直到达到您设置的最大数量。请参考[定价页面](https://cloud.google.com/vertex-ai/pricing#prediction-prices)了解使用多个节点进行自动扩展的成本。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

# The name of the job
BATCH_PREDICTION_JOB_NAME = "cifar10_batch_prediction_unique"

# Folder in the bucket to write results to
DESTINATION_FOLDER = "batch_prediction_results"

# The Cloud Storage bucket to upload results to
BATCH_PREDICTION_GCS_DEST_PREFIX = BUCKET_URI + "/" + DESTINATION_FOLDER

# Make SDK batch_predict method call
batch_prediction_job = model.batch_predict(
    instances_format="jsonl",
    predictions_format="jsonl",
    job_display_name=BATCH_PREDICTION_JOB_NAME,
    gcs_source=BATCH_PREDICTION_GCS_SOURCE,
    gcs_destination_prefix=BATCH_PREDICTION_GCS_DEST_PREFIX,
    model_parameters=None,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    machine_type="n1-standard-4",
    sync=True,
)

### 获取批量预测结果
当批量预测处理完成时，您可以最终查看存储在您设置为输出的云存储路径上的预测结果。预测结果将以JSONL格式存在，这是您在创建批量预测作业时指定的格式。预测结果位于以名称prediction开头的子目录中。在该目录中，有一个名为prediction.results-xxxx-of-xxxx的文件。

让我们显示内容。您将为每个预测获得一行。该行是对应的CIFAR10类的softmax概率分布。

In [ ]:
RESULTS_DIRECTORY = "prediction_results"
RESULTS_DIRECTORY_FULL = RESULTS_DIRECTORY + "/" + DESTINATION_FOLDER

# Create missing directories
os.makedirs(RESULTS_DIRECTORY, exist_ok=True)

# Get the Cloud Storage paths for each result
! gsutil -m cp -r $BATCH_PREDICTION_GCS_DEST_PREFIX $RESULTS_DIRECTORY

# Get most recently modified directory
latest_directory = max(
    (
        os.path.join(RESULTS_DIRECTORY_FULL, d)
        for d in os.listdir(RESULTS_DIRECTORY_FULL)
    ),
    key=os.path.getmtime,
)

# Get downloaded results in directory
results_files = []
for dirpath, subdirs, files in os.walk(latest_directory):
    for file in files:
        if file.startswith("prediction.results"):
            results_files.append(os.path.join(dirpath, file))

# Consolidate all the results into a list
results = []
for results_file in results_files:
    # Download each result
    with open(results_file, "r") as file:
        results.extend([json.loads(line) for line in file.readlines()])

### 评估结果

然后，您可以对预测结果进行快速评估：

1. `np.argmax`：将每个置信度列表转换为标签
2. 比较预测的标签与实际标签
3. 将`正确/总数`计算为`准确率`

为了提高准确率，请尝试训练更多个周期。

In [ ]:
y_predicted = [np.argmax(result["prediction"]) for result in results]

correct = sum(y_predicted == np.array(y_test))
accuracy = len(y_predicted)
print(
    f"Correct predictions = {correct}, Total predictions = {accuracy}, Accuracy = {correct/accuracy}"
)

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的个别资源：

- 训练作业
- 模型
- 云存储桶

In [ ]:
# Warning: Setting this to true will delete everything in your bucket
delete_bucket = False

# Delete the training job
job.delete()

# Delete the model
model.delete()

if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI